In [19]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import Dict

In [20]:
books = pd.read_json("data/books.json").values

In [71]:
books[3][0]["ask_volumes"]

[78, 10000, 20]

In [112]:
# input_a_dictionary
def getPriceVolArr(book):
    book = book[0]
    askVolPriceProd = np.array([])
    bidVolPriceProd = np.array([])
    

    askVolPriceProd = np.concatenate(
            (askVolPriceProd,
             [book["ask_prices"] for t in book["ask_volumes"]]),
            axis=None
        )
            
    bidVolPriceProd = np.concatenate(
            (bidVolPriceProd,
            [book["bid_prices"] for t in book["bid_volumes"]]),
            axis=None
        )
    return askVolPriceProd, bidVolPriceProd


In [113]:
ask_Var = np.empty(len(books))
bid_Var = np.empty(len(books))
ask_mn = np.empty(len(books))
bid_mn = np.empty(len(books))
for i in range(len(books)):
    current_book = books[i]
    askVPP, bidVPP = getPriceVolArr(current_book)
    ask_Var[i] = np.var(askVPP) if len(askVPP) !=0 else 0
    ask_mn[i] = np.mean(askVPP) if len(askVPP) !=0 else 0
    bid_Var[i] = np.var(bidVPP) if len(bidVPP) !=0 else 0
    bid_mn[i] = np.mean(bidVPP) if len(bidVPP) !=0 else 0


{'ask_prices': [], 'ask_volumes': [], 'bid_prices': [], 'bid_volumes': []}

In [114]:
ask_Var

array([      0.        ,       0.        ,   90000.        ,
        606666.66666667,  462500.        ,  482222.22222222,
        197600.        ,   34400.        ,   58400.        ,
         77600.        ,   52000.        ,   52000.        ,
        202400.        ,  173600.        ,  140000.        ,
        161600.        ,   94400.        ,  140000.        ,
         85600.        ,   70400.        ,   70400.        ,
         53600.        ,   42400.        ,   46400.        ,
         34400.        ,   46400.        ,   34400.        ,
         52000.        ,   34400.        ,   34400.        ,
        154400.        ,  117600.        ,   45600.        ,
         60000.        ,   42400.        ,   61600.        ,
         61600.        ,   61600.        ,   42400.        ,
         68000.        ,   20000.        ,   20000.        ,
         20000.        ,   34400.        ,   77600.        ,
        128000.        ,  147500.        ,   42400.        ,
        121600.        ,